In [1]:
!pip install yfinance --quiet

In [2]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [90]:
ticker = '^BVSP'
dados = yf.download(ticker, start='2005-01-01', interval='1d')

[*********************100%***********************]  1 of 1 completed


In [91]:
period = 20
dados['Sma'] = dados['Close'].rolling(period).mean()

In [92]:
dados['Afastado_media_alta'] = dados['Low'] > dados['Sma']
dados['Afastado_media_baixa'] = dados['High'] < dados['Sma']

In [93]:
contagem = []
count = 0

for i in range(len(dados)):
  if dados['Afastado_media_alta'][i] == True:
    count += 1
    contagem.append(count)

  else:
    count = 0
    contagem.append(count)

dados['Contagem_alta'] = contagem
  

In [94]:
contagem = []
count = 0

for i in range(len(dados)):
  if dados['Afastado_media_baixa'][i] == True:
    count += 1
    contagem.append(count)

  else:
    count = 0
    contagem.append(count)

dados['Contagem_baixa'] = contagem
  

In [103]:
df_figure = dados.loc['2008-01-01':'2009-01-01']
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Candlestick(name=ticker, x=df_figure.index, open=df_figure['Open'],
                             high=df_figure['High'], low=df_figure['Low'], 
                             close=df_figure['Close']), row=1, col=1)

fig.add_trace(go.Scatter(name='SMA_20', x=df_figure.index, y=df_figure['Sma'],
                         marker_color='gray'), row=1, col=1)

fig.add_trace(go.Scatter(name='Count_afastado_media', x=df_figure.index, 
                         y=(1 + .01)*df_figure["High"].where(df_figure["Afastado_media_alta"] == True),
                         mode='markers', marker=dict(color='Green', size=5)))

fig.add_trace(go.Scatter(name='Count_afastado_baixa', x=df_figure.index, 
                         y=(1 - .01)*df_figure["Low"].where(df_figure["Afastado_media_baixa"] == True),
                         mode='markers', marker=dict(color='Red', size=5)))

fig.add_trace(go.Scatter(name='afastamento_alta', x=df_figure.index, y=df_figure['Contagem_alta'],
                         marker_color='green'), row=2, col=1)

fig.add_trace(go.Scatter(name='afastamento_baixa', x=df_figure.index, y=df_figure['Contagem_baixa'],
                         marker_color='red'), row=2, col=1)



fig.update_layout(title_text='<b>Tempo afastado da média', template='plotly', 
                  margin=dict(l=50, r=20, t=70, b=20), width=1400, height=700,
                  xaxis_rangeslider_visible=False)

fig.show()

